In [95]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional,Conv1D, MaxPooling1D, Flatten, BatchNormalization, Input, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model
import re

In [2]:
df_phising = pd.read_csv(r"C:\Users\z004z92k\Downloads\phising_email\phishing_email.csv")

In [3]:
df_phising.head()

,text_combined,label
0,hpl nom may 25 2001 see attached file hplno 52...,0
1,nom actual vols 24 th forwarded sabrae zajac h...,0
2,enron actuals march 30 april 1 201 estimated a...,0
3,hpl nom may 30 2001 see attached file hplno 53...,0
4,hpl nom june 1 2001 see attached file hplno 60...,0


In [4]:
df_Nigerian = pd.read_csv(r"C:\Users\z004z92k\Downloads\phising_email\Nigerian_Fraud.csv")

In [5]:
df_Nigerian.head()

,sender,receiver,date,subject,body,urls,label
0,MR. JAMES NGOLA. <james_ngola2002@maktoob.com>,webmaster@aclweb.org,"Thu, 31 Oct 2002 02:38:20 +0000",URGENT BUSINESS ASSISTANCE AND PARTNERSHIP,FROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-2...,0,1
1,Mr. Ben Suleman <bensul2004nng@spinfinder.com>,R@M,"Thu, 31 Oct 2002 05:10:00 -0000",URGENT ASSISTANCE /RELATIONSHIP (P),"Dear Friend,\n\nI am Mr. Ben Suleman a custom ...",0,1
2,PRINCE OBONG ELEME <obong_715@epatra.com>,webmaster@aclweb.org,"Thu, 31 Oct 2002 22:17:55 +0100",GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...,0,1
3,PRINCE OBONG ELEME <obong_715@epatra.com>,webmaster@aclweb.org,"Thu, 31 Oct 2002 22:44:20 -0000",GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...,0,1
4,Maryam Abacha <m_abacha03@www.com>,R@M,"Fri, 01 Nov 2002 01:45:04 +0100",I Need Your Assistance.,"Dear sir, \n \nIt is with a heart full of hope...",0,1


In [6]:
df_SpamAssasin = pd.read_csv(r"C:\Users\z004z92k\Downloads\phising_email\SpamAssasin.csv")

In [7]:
df_SpamAssasin.head()

,sender,receiver,date,subject,body,label,urls
0,Robert Elz <kre@munnari.OZ.AU>,Chris Garrigues <cwg-dated-1030377287.06fa6d@D...,"Thu, 22 Aug 2002 18:26:25 +0700",Re: New Sequences Window,"Date: Wed, 21 Aug 2002 10:54:46 -0500 ...",0,1
1,Steve Burt <Steve_Burt@cursor-system.com>,"""'zzzzteana@yahoogroups.com'"" <zzzzteana@yahoo...","Thu, 22 Aug 2002 12:46:18 +0100",[zzzzteana] RE: Alexander,"Martin A posted:\nTassos Papadopoulos, the Gre...",0,1
2,"""Tim Chapman"" <timc@2ubh.com>",zzzzteana <zzzzteana@yahoogroups.com>,"Thu, 22 Aug 2002 13:52:38 +0100",[zzzzteana] Moscow bomber,Man Threatens Explosion In Moscow \n\nThursday...,0,1
3,Monty Solomon <monty@roscom.com>,undisclosed-recipient: ;,"Thu, 22 Aug 2002 09:15:25 -0400",[IRR] Klez: The Virus That Won't Die,Klez: The Virus That Won't Die\n \nAlready the...,0,1
4,Stewart Smith <Stewart.Smith@ee.ed.ac.uk>,zzzzteana@yahoogroups.com,"Thu, 22 Aug 2002 14:38:22 +0100",Re: [zzzzteana] Nothing like mama used to make,"> in adding cream to spaghetti carbonara, whi...",0,1


In [8]:
df_Ling = pd.read_csv(r"C:\Users\z004z92k\Downloads\phising_email\Ling.csv")
df_Ling.head()

,subject,body,label
0,job posting - apple-iss research center,content - length : 3386 apple-iss research cen...,0
1,NaN,"lang classification grimes , joseph e . and ba...",0
2,query : letter frequencies for text identifica...,i am posting this inquiry for sergei atamas ( ...,0
3,risk,a colleague and i are researching the differin...,0
4,request book information,earlier this morning i was on the phone with a...,0


In [10]:
df_Nazario = pd.read_csv(r"C:\Users\z004z92k\Downloads\phising_email\Nazario.csv")
df_Nazario.head()

,sender,receiver,date,subject,body,urls,label
0,Mail System Internal Data <MAILER-DAEMON@monke...,NaN,28 Sep 2017 09:57:25 -0400,DON'T DELETE THIS MESSAGE -- FOLDER INTERNAL DATA,This text is part of the internal format of yo...,1,1
1,cPanel <service@cpanel.com>,jose@monkey.org,"Fri, 30 Oct 2015 00:00:48 -0500",Verify Your Account,Business with \t\t\t\t\t\t\t\tcPanel & WHM \t...,1,1
2,Microsoft Outlook <recepcao@unimedceara.com.br>,NaN,"Fri, 30 Oct 2015 06:21:59 -0300 (BRT)",Helpdesk Mailbox Alert!!!,Your two incoming mails were placed on pending...,1,1
3,Ann Garcia <AnGarcia@mcoe.org>,"""info@maaaaa.org"" <info@maaaaa.org>","Fri, 30 Oct 2015 14:54:33 +0000",IT-Service Help Desk,Password will expire in 3 days. Click Here To ...,0,1
4,"""USAA"" <usaaacctupdate@sccu4u.com>",Recipients <usaaacctupdate@sccu4u.com>,"Fri, 30 Oct 2015 14:02:33 -0500",Final USAA Reminder - Update Your Account Now,"To ensure delivery to your inbox, please add U...",1,1


In [11]:
df_enron = pd.read_csv(r"C:\Users\z004z92k\Downloads\phising_email\Enron.csv")

In [12]:
df_enron.head()

,subject,body,label
0,"hpl nom for may 25 , 2001",( see attached file : hplno 525 . xls )\r\n- h...,0
1,re : nom / actual vols for 24 th,- - - - - - - - - - - - - - - - - - - - - - fo...,0
2,"enron actuals for march 30 - april 1 , 201","estimated actuals\r\nmarch 30 , 2001\r\nno flo...",0
3,"hpl nom for may 30 , 2001",( see attached file : hplno 530 . xls )\r\n- h...,0
4,"hpl nom for june 1 , 2001",( see attached file : hplno 601 . xls )\r\n- h...,0


In [13]:
df_ceas = pd.read_csv(r"C:\Users\z004z92k\Downloads\phising_email\CEAS_08.csv")
df_ceas.head()

,sender,receiver,date,subject,body,label,urls
0,Young Esposito <Young@iworld.de>,user4@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 16:31:02 -0700",Never agree to be a loser,"Buck up, your troubles caused by small dimensi...",1,1
1,Mok <ipline's1983@icable.ph>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 18:31:03 -0500",Befriend Jenna Jameson,\nUpgrade your sex and pleasures with these te...,1,1
2,Daily Top 10 <Karmandeep-opengevl@universalnet...,user2.9@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 20:28:00 -1200",CNN.com Daily Top 10,>+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+=+...,1,1
3,Michael Parker <ivqrnai@pobox.com>,SpamAssassin Dev <xrh@spamassassin.apache.org>,"Tue, 05 Aug 2008 17:31:20 -0600",Re: svn commit: r619753 - in /spamassassin/tru...,Would anyone object to removing .so from this ...,0,1
4,Gretchen Suggs <externalsep1@loanofficertool.com>,user2.2@gvc.ceas-challenge.cc,"Tue, 05 Aug 2008 19:31:21 -0400",SpecialPricesPharmMoreinfo,\nWelcomeFastShippingCustomerSupport\nhttp://7...,1,1


In [14]:
dataset = pd.concat([df_Nigerian, df_SpamAssasin, df_Nazario, df_ceas], axis=0, ignore_index=True)

In [15]:
dataset.head()

,sender,receiver,date,subject,body,urls,label
0,MR. JAMES NGOLA. <james_ngola2002@maktoob.com>,webmaster@aclweb.org,"Thu, 31 Oct 2002 02:38:20 +0000",URGENT BUSINESS ASSISTANCE AND PARTNERSHIP,FROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-2...,0,1
1,Mr. Ben Suleman <bensul2004nng@spinfinder.com>,R@M,"Thu, 31 Oct 2002 05:10:00 -0000",URGENT ASSISTANCE /RELATIONSHIP (P),"Dear Friend,\n\nI am Mr. Ben Suleman a custom ...",0,1
2,PRINCE OBONG ELEME <obong_715@epatra.com>,webmaster@aclweb.org,"Thu, 31 Oct 2002 22:17:55 +0100",GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...,0,1
3,PRINCE OBONG ELEME <obong_715@epatra.com>,webmaster@aclweb.org,"Thu, 31 Oct 2002 22:44:20 -0000",GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...,0,1
4,Maryam Abacha <m_abacha03@www.com>,R@M,"Fri, 01 Nov 2002 01:45:04 +0100",I Need Your Assistance.,"Dear sir, \n \nIt is with a heart full of hope...",0,1


In [19]:
print(dataset.loc[dataset['receiver'] == 'user4@gvc.ceas-challenge.cc'])

                                                  sender  \
10706                   Young Esposito <Young@iworld.de>   
10737                 Cara Childs <xigoing@cityofws.org>   
10834       Marcus Trujillo <nvibqkcie@bobbittville.com>   
10909                  Freida Alfaro <Freida@sbc.edu.sg>   
10922                                           bobot <>   
...                                                  ...   
49267  Luann Bland <confectionerstdcj84@cad-netbank.com>   
49529   Janice Frost <florinze469@danielspublishing.com>   
49644                  Elise Sumner <Elise@cybercom.net>   
49726               Robin Schneider <Robin@ac-verlag.de>   
49762     Corrina Hosseini <Corrina-eskytykn@foxcub.org>   

                          receiver                             date  \
10706  user4@gvc.ceas-challenge.cc  Tue, 05 Aug 2008 16:31:02 -0700   
10737  user4@gvc.ceas-challenge.cc  Tue, 05 Aug 2008 18:32:47 -0500   
10834  user4@gvc.ceas-challenge.cc  Tue, 05 Aug 2008 20:37:46 -030

In [20]:
dataset.shape

(49860, 7)

In [21]:
dataset.dropna(inplace=True)

In [22]:
dataset.shape

(47290, 7)

In [43]:
def separate_senderdomain(text):
    domain = re.findall(r'<(.*?)>', text)

    if not domain:
        domain.append('unknown')
    return ''.join(domain)

In [44]:
dataset['sender_domain']=dataset['sender'].apply(separate_senderdomain)

In [45]:
print(dataset['sender_domain'].loc[2])

obong_715@epatra.com


In [34]:
def extract_url(text):

    url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_pattern, text)
    text_without_urls = re.sub(url_pattern, 'url', text)
    return text_without_urls,urls


In [35]:
dataset['body'], dataset['extracted_url'] = zip(*dataset['body'].apply(extract_url))

In [41]:
print(dataset.loc[dataset['urls'] == 1])

                                        sender  \
27             obina okoro <obinaokoro@37.com>   
31               mbeki ngumeni <mbengu@37.com>   
40           deborah kabila <kabila72b@37.com>   
41                 Khalifa Sese <kss07@go.com>   
45     Oliveira Savimbi <olivsav@netscape.net>   
...                                        ...   
49849      Wendy Richter <gerich@miffbank.com>   
49850       Evan Herrera <Evan@uscellular.com>   
49851   Amandalee <tamekut_1991@smarttech.com>   
49853    Nick Zeljkovic <kppyozizjt@site5.com>   
49859      Christian Heimes <wluhe@cheimes.de>   

                                  receiver                             date  \
27                       obinaokoro@37.com  Mon, 25 Nov 2002 09:00:25 -0800   
31                           mbengu@37.com  Thu, 28 Nov 2002 08:34:54 -0800   
40                        kabila72b@37.com  Mon, 09 Dec 2002 05:10:49 -0800   
41                       colibri@let.uu.nl  Wed, 11 Dec 2002 05:23:51 -0800   
45  

In [46]:
dataset.head()

,sender,receiver,date,subject,body,urls,label,sender_domain,extracted_url
0,MR. JAMES NGOLA. <james_ngola2002@maktoob.com>,webmaster@aclweb.org,"Thu, 31 Oct 2002 02:38:20 +0000",URGENT BUSINESS ASSISTANCE AND PARTNERSHIP,FROM:MR. JAMES NGOLA.\nCONFIDENTIAL TEL: 233-2...,0,1,james_ngola2002@maktoob.com,[]
1,Mr. Ben Suleman <bensul2004nng@spinfinder.com>,R@M,"Thu, 31 Oct 2002 05:10:00 -0000",URGENT ASSISTANCE /RELATIONSHIP (P),"Dear Friend,\n\nI am Mr. Ben Suleman a custom ...",0,1,bensul2004nng@spinfinder.com,[]
2,PRINCE OBONG ELEME <obong_715@epatra.com>,webmaster@aclweb.org,"Thu, 31 Oct 2002 22:17:55 +0100",GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...,0,1,obong_715@epatra.com,[]
3,PRINCE OBONG ELEME <obong_715@epatra.com>,webmaster@aclweb.org,"Thu, 31 Oct 2002 22:44:20 -0000",GOOD DAY TO YOU,FROM HIS ROYAL MAJESTY (HRM) CROWN RULER OF EL...,0,1,obong_715@epatra.com,[]
4,Maryam Abacha <m_abacha03@www.com>,R@M,"Fri, 01 Nov 2002 01:45:04 +0100",I Need Your Assistance.,"Dear sir, \n \nIt is with a heart full of hope...",0,1,m_abacha03@www.com,[]


In [48]:
dataset['num_urls'] = dataset['extracted_url'].apply(len)

In [49]:
dataset['body_len'] = dataset['body'].apply(len)

In [54]:
# functions to extract features from URL
from urllib.parse import urlparse # Number of subdomains in the URL
def extract_url_features(url):
    features = []

    features.append(1 if url.lower().startswith("https") else 0)

    features.append(len(url))

    try:
        parsed_url=urlparse(url)
        subdomain_count = len(parsed_url.netloc.split('.')) -2 #subdomains count, excluding domain and top level domain
        features.append(subdomain_count)
    except Exception as e:
        features.append(0)

    suspicious_keywords = ["login", "verify", "account", "secure", "update","password"]
    features.append(1 if any(keyword in url.lower() for keyword in suspicious_keywords) else 0)

    features.append(url.count('?'))

    return features

def extract_features_from_urls(urls):
    all_features =[]

    for url in urls:
        features = extract_url_features(url)
        all_features.append(features)

    if all_features:
        aggregated_features = []
        for i in range(len(all_features[0])):
            if isinstance(all_features[0][i], (int,float)):
                aggregated_features.append(sum(f[i] for f in all_features) / len(all_features))
            else:
                aggregated_features.append(None)
        return aggregated_features
    else:
        return [0*5]

dataset['extracted_url'] = dataset['extracted_url'].apply(extract_features_from_urls)
    

In [55]:
dataset.loc[dataset['urls'] == 1].head()

,sender,receiver,date,subject,body,urls,label,sender_domain,extracted_url,num_urls,body_len
27,obina okoro <obinaokoro@37.com>,obinaokoro@37.com,"Mon, 25 Nov 2002 09:00:25 -0800",urgent reply,"ATTN:President/Ceo Business Proposal, > > DEA...",1,1,obinaokoro@37.com,"[0.0, 21.0, 0.6666666666666666, 0.0, 0.0]",3,2737
31,mbeki ngumeni <mbengu@37.com>,mbengu@37.com,"Thu, 28 Nov 2002 08:34:54 -0800",STRICTLY CONFIDENTIAL & URGENT,"Dear sir, I am Dr.Mbeki Ngumeni, a native of...",1,1,mbengu@37.com,"[0.0, 21.0, 0.6666666666666666, 0.0, 0.0]",3,3858
40,deborah kabila <kabila72b@37.com>,kabila72b@37.com,"Mon, 09 Dec 2002 05:10:49 -0800",hi friend,"Dear friend, RE save our soul I am Mrs. De...",1,1,kabila72b@37.com,"[0.0, 21.0, 0.6666666666666666, 0.0, 0.0]",3,2664
41,Khalifa Sese <kss07@go.com>,colibri@let.uu.nl,"Wed, 11 Dec 2002 05:23:51 -0800",ATTENTION,REPLY TO : princekhalifa_mss2009@yahoo.co.uk\n...,1,1,kss07@go.com,"[0.0, 18.0, 1.0, 0.0, 0.0]",1,2233
45,Oliveira Savimbi <olivsav@netscape.net>,webmaster@aclweb.org,"Tue, 24 Dec 2002 16:52:18 -0000",PRIVATE EMAIL MESSAGE,"Good Day,\n\nWith warm heart I offer my friend...",1,1,olivsav@netscape.net,"[0.0, 43.0, 1.0, 0.0, 0.0]",1,4388


In [56]:
#clean body and subject
def clean_text(text):
    text = re.sub(r'<.*?>','',text)
    text = re.sub(r'http\S+|www\S+','',text)
    text = re.sub(r'[^\w\s]','',text)
    text = text.lower().strip()
    return text

In [57]:
dataset['clean_body'] = dataset['body'].apply(clean_text)
dataset['clean_subject'] = dataset['subject'].apply(clean_text)

In [60]:
dataset = dataset.drop(columns = ['sender','date','subject','body'])

KeyError: "['sender', 'date', 'subject', 'body'] not found in axis"

In [61]:
dataset = dataset.drop(columns = ['receiver'])

In [64]:
dataset.head()

,urls,label,sender_domain,extracted_url,num_urls,body_len,clean_body,clean_subject
0,0,1,james_ngola2002@maktoob.com,[0],0,3386,frommr james ngola\nconfidential tel 233275879...,urgent business assistance and partnership
1,0,1,bensul2004nng@spinfinder.com,[0],0,3247,dear friend\n\ni am mr ben suleman a custom of...,urgent assistance relationship p
2,0,1,obong_715@epatra.com,[0],0,3257,from his royal majesty hrm crown ruler of elem...,good day to you
3,0,1,obong_715@epatra.com,[0],0,3255,from his royal majesty hrm crown ruler of elem...,good day to you
4,0,1,m_abacha03@www.com,[0],0,4024,dear sir \n \nit is with a heart full of hope ...,i need your assistance


In [65]:
def remove_slashN(text):
    text = text.replace('\n',' ')
    return text

dataset['clean_body'] = dataset['clean_body'].apply(remove_slashN)

In [66]:
dataset.head()

,urls,label,sender_domain,extracted_url,num_urls,body_len,clean_body,clean_subject
0,0,1,james_ngola2002@maktoob.com,[0],0,3386,frommr james ngola confidential tel 2332758790...,urgent business assistance and partnership
1,0,1,bensul2004nng@spinfinder.com,[0],0,3247,dear friend i am mr ben suleman a custom offi...,urgent assistance relationship p
2,0,1,obong_715@epatra.com,[0],0,3257,from his royal majesty hrm crown ruler of elem...,good day to you
3,0,1,obong_715@epatra.com,[0],0,3255,from his royal majesty hrm crown ruler of elem...,good day to you
4,0,1,m_abacha03@www.com,[0],0,4024,dear sir it is with a heart full of hope th...,i need your assistance


In [68]:
dataset['clean_body_len'] = dataset['clean_body'].apply(len)

In [69]:
dataset['combined_text'] = dataset['sender_domain'] + " " + dataset['clean_subject'] + " " +dataset['clean_body']

In [99]:
#from sklearn.preprocessing import LabelEncoder

#encoder = LabelEncoder()
#dataset['extracted_url_encoded'] = encoder.fit_transform(dataset['extracted_url'].astype(str))
#extracted_url = dataset['extracted_url_encoded'].values.reshape(-1, 1)  # Use this in training

In [71]:
texts = dataset['combined_text'].values
labels = dataset['label'].values

In [72]:
texts.shape

(47290,)

In [74]:
num_urls = dataset['num_urls'].values.reshape(-1, 1)
body_len = dataset['body_len'].values.reshape(-1, 1)

In [75]:
num_urls.shape

(47290, 1)

In [101]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
dataset['extracted_url_encoded'] = encoder.fit_transform(dataset['extracted_url'].astype(str))
extracted_url = dataset['extracted_url_encoded'].values.reshape(-1, 1)  # Use this in training

In [81]:
import matplotlib.pyplot as plt

vocab_size = 10000
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
text_length = [len(seq) for seq in sequences]
max_length = np.percentile(text_length,95)
print(max_length)

659.0


In [82]:
X_padded = pad_sequences(sequences, maxlen=int(max_length), padding='post', truncating='post')

In [102]:
X_train_text, X_test_text, y_train, y_test = train_test_split(X_padded, labels, test_size=0.2, random_state=42)
X_train_meta, X_test_meta = train_test_split(num_urls, test_size=0.2, random_state=42)
X_train_len, X_test_len = train_test_split(body_len, test_size=0.2, random_state=42)
X_train_url, X_test_url = train_test_split(extracted_url, test_size=0.2, random_state=42)

In [107]:
print(X_train_text.shape)
print(X_train_meta.shape)
print(X_train_len.shape)
print(X_train_url.shape)

(37832, 659)
(37832, 1)
(37832, 1)
(37832, 1)


In [91]:
text_input = Input(shape=(int(max_length),))
embedding = Embedding(vocab_size, 32, input_length=max_length)(text_input)
conv = Conv1D(64, 5, activation='relu', padding='same')(embedding)
pool = MaxPooling1D(2)(conv)
norm = BatchNormalization()(pool)
lstm = Bidirectional(LSTM(32, return_sequences=True, kernel_regularizer=l2(0.01)))(norm)
dropout1 = Dropout(0.5)(lstm)
lstm2 = Bidirectional(LSTM(16, kernel_regularizer=l2(0.01)))(dropout1)

In [103]:
num_input = Input(shape=(1,))
len_input = Input(shape=(1,))
url_input = Input(shape=(1,))

In [104]:
concat = Concatenate()([lstm2, num_input, len_input, url_input])
dense1 = Dense(32, activation='relu')(concat)
dropout2 = Dropout(0.5)(dense1)
output = Dense(1, activation='sigmoid')(dropout2)

model = Model(inputs=[text_input, num_input, len_input, url_input], outputs=output)

In [105]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# 6. Train the Model
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
history = model.fit(
    [X_train_text, X_train_meta, X_train_len, X_train_url], y_train,
    epochs=10,
    batch_size=32,
    validation_split=0.2,
    callbacks=[early_stop],
    verbose=2
)


Epoch 1/10
946/946 - 195s - 206ms/step - accuracy: 0.7077 - loss: 60.2114 - val_accuracy: 0.9604 - val_loss: 1.3835
Epoch 2/10
946/946 - 219s - 231ms/step - accuracy: 0.8978 - loss: 1.5094 - val_accuracy: 0.8688 - val_loss: 0.5896
Epoch 3/10
946/946 - 212s - 225ms/step - accuracy: 0.9002 - loss: 0.8859 - val_accuracy: 0.9738 - val_loss: 0.2891
Epoch 4/10
946/946 - 212s - 224ms/step - accuracy: 0.9028 - loss: 0.4622 - val_accuracy: 0.9769 - val_loss: 0.2409
Epoch 5/10
946/946 - 233s - 246ms/step - accuracy: 0.8942 - loss: 0.3565 - val_accuracy: 0.9175 - val_loss: 0.2146
Epoch 6/10
946/946 - 241s - 254ms/step - accuracy: 0.9076 - loss: 0.2959 - val_accuracy: 0.9697 - val_loss: 0.1580
Epoch 7/10
946/946 - 208s - 220ms/step - accuracy: 0.9217 - loss: 0.2699 - val_accuracy: 0.8430 - val_loss: 0.6717
Epoch 8/10
946/946 - 216s - 228ms/step - accuracy: 0.9419 - loss: 0.2540 - val_accuracy: 0.9856 - val_loss: 0.1141
Epoch 9/10
946/946 - 206s - 218ms/step - accuracy: 0.9468 - loss: 0.1811 - val_

In [108]:
loss, accuracy = model.evaluate([X_test_text, X_test_meta, X_test_len, X_test_url], y_test, verbose=2)
print(f'Final Test Accuracy: {accuracy * 100:.2f}%')


296/296 - 13s - 44ms/step - accuracy: 0.9851 - loss: 0.0815
Final Test Accuracy: 98.51%


In [113]:
def predict_email(text, num_urls_value, body_len_value, extracted_url_value):
    clean_sen = clean_text(text)
    seq = tokenizer.texts_to_sequences([clean_sen])
    padded = pad_sequences(seq, maxlen=int(max_length), padding='post', truncating='post')
    prediction = model.predict([padded, np.array([[num_urls_value]]), np.array([[body_len_value]]), np.array([[extracted_url_value]])])[0][0]
    return 'Spam/Phishing' if prediction > 0.5 else 'Not Spam'

# Test the function
print(predict_email("Congratulations! You have won a lottery. Click here to claim your prize!", 2, 150, 1))
print(predict_email("Meeting scheduled for tomorrow at 10 AM.", 0, 50, 0))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
Spam/Phishing
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Spam/Phishing
